In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import mysql.connector
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
jum_su = []
v_review = []
b_review = []
s_review = []
name = []
var = []
add = []
gu_l = []
num = []

In [3]:
driver = webdriver.Chrome(service=Service("../craw/driver/chromedriver"))
driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")


In [4]:
gu = ['강남구','강동구','강북구','강서구','중구','성동구','용산구','은평구','동작구','도봉구','송파구','성북구','서초구','양천구','종로구','마포구',\
      '중랑구','노원구','금천구','관악구','광진구','구로구','영등포구']
wait = WebDriverWait(driver, 1)  # 25개 각 구 별로 네이버 지도에서 맛집을 검색하여 정보를 가져온다.
for i in gu:
      driver.switch_to.default_content() #네이버 지도 특성에 맞추어 기본프레임을 먼저 드라이버로 찾아낸 뒤에 검색결과 프레임을 오갈 수 있다.
      mat = driver.find_element(By.CLASS_NAME,'input_box') # 검색어 입력창의 element를 찾기
      mat.click()
      time.sleep(1.5)
      sear = mat.find_element(By.CLASS_NAME,'input_search') # 입력창에 텍스트를 입력하는 부분찾기
      sear.clear() # 앞선 검색어를 지워주기위해 clear
      sear.send_keys(i+' 맛집') # 검색어 입력
      sear.send_keys(Keys.RETURN) # Enter키로 검색실행
      driver.implicitly_wait(3) # 화면이 로딩될때가지 최대 3초 대기
      driver.switch_to.frame('searchIframe') # 검색결과(음식점 목록을 나타내는 프레임을 찾기)
      time.sleep(1)
      mok = driver.find_element(By.CLASS_NAME,'Ryr1F')  # 검색결과 목록
      lis = mok.find_elements(By.TAG_NAME,'li') # 목록에서 순서별로 찾을 준비
      count = 0 # 현재 진행하는 검색목록의 index를 따로 표시하기 위한 count변수
      count2 = 9 # 네이버 검색결과가 최대 10개까지만 표기되고 새롭게 추가갱신 되는 방식이기 때문에 9번째마다 갱신할 시간을 주고,
                 # 다시한번 드라이버가 가지고 있는 전체 리스트를 갱신하기 위해 count2변수 
      count3 = 1 # 페이지를 세기 위한 카운트
      n = 1 # 각 식당마다 정보가 제각각이기 때문에 정보를 가져오다가 문제가 발생할 경우 어느 위치에서 문제가 발생하는지를 파악하기위한 넘버링용 변수
      while lis[count] != 'null': # 리스트가 갱신되는 형식이기 때문에 for문을 사용할 수 없고 식당목록이 갱신되는한 계속해서 반복하는 while문을 사용
            num.append(n) # 데이터 분석을 용이하게 하기위한 넘버링
            n += 1 #넘버링 카운터 증가
            gu_l.append(i)
            title = lis[count].find_element(By.CLASS_NAME,'N_KDL')
            wait.until(EC.element_to_be_clickable(title)) # element요소가 클릭이 가능해질때까지 대기
            title.click()
            time.sleep(1.5)
            driver.switch_to.default_content() # switch_to.default_content()로 전환해야 frame_in iframe을 제대로 잡을 수 있다.
            try:
                  wait.until(EC.frame_to_be_available_and_switch_to_it(driver.switch_to.default_content())) 
                  # 프레임 전환이 가능해질 때까지 대기해주는 함수
            except:
                 time.sleep(2)
            driver.switch_to.frame("entryIframe") # 검색결과에서 세부사항을 보기위한 entryframe으로 이동
            try:
                  wait.until(EC.element_to_be_selected(driver.find_element(By.CLASS_NAME,'GHAhO')))
            except:
                  time.sleep(2)
            nam = driver.find_element(By.CLASS_NAME,'GHAhO') #가게이름
            name.append(nam.text)
            vari = driver.find_element(By.CLASS_NAME,'lnJFt') #식당종류
            var.append(vari.text)
            ad_f = driver.find_element(By.CLASS_NAME,'LDgIH') #주소
            add.append(ad_f.text)
            bul = driver.find_element(By.CLASS_NAME,'dAsGb') # 리뷰전체
            bul_l = bul.find_elements(By.TAG_NAME,'a') # 방문자와 블로그 리뷰 분리
            if 'class="PXMot LXIwF"' in bul.get_attribute('innerHTML'): # 별점이 있을경우
                  jum = bul.find_element(By.CLASS_NAME,'PXMot.LXIwF') #별점 element 의 클래스네임
                  point = jum.get_attribute('innerText').split() #별점만 분리
                  jum_su.append(float(point[1])) # 소수형으로 변환하여 저장
                  try:
                        rev1 = bul_l[0].get_attribute('innerText').split()  # 방문자 리뷰
                        rev1[-1] = int(rev1[-1].replace(',','')) # 천단위가 넘어갈 경우 쉼표빼고 int형으로 변환
                        v_review.append(rev1[-1]) # 방문자 리뷰 점수 추가
                        rev2 = bul_l[1].get_attribute('innerText').split() #  블로그 리뷰 점수
                        rev2[-1] = int(rev2[-1].replace(',','')) # 형변환
                        b_review.append(rev2[-1]) # 블로그 리뷰점수 추가
                        s_review.append(rev1[-1]+rev2[-1]) #방문자와 블로그 리뷰 수를 더해서 총 리뷰수를 계산
                  except:
                        rev = bul_l[0].get_attribute('innerText').split()
                        rev[-1] = int(rev[-1].replace(',',''))
                        if v_review[n-2] == rev[-1]:
                              b_review.append(0)
                        else:
                              b_review.append(0)
                        s_review.append(rev[-1])
            else:
                  try: # 블로그 리뷰나 방문자 리뷰 둘중 하나가 없을 경우 무시하고 하나값을 총 리뷰수로 취급
                        jum_su.append(0) # 별점이 없을경우 0점
                        rev1 = bul_l[0].get_attribute('innerText').split()
                        rev1[-1] = int(rev1[-1].replace(',',''))
                        v_review.append(rev1[-1])
                        rev2 = bul_l[1].get_attribute('innerText').split()
                        rev2[-1] = int(rev2[-1].replace(',',''))
                        b_review.append(rev2[-1])
                        s_review.append(rev1[-1]+rev2[-1])
                  except:
                        rev = bul_l[0].get_attribute('innerText').split()
                        rev[-1] = int(rev[-1].replace(',',''))
                        if v_review[n-2] == rev[-1]:
                              b_review.append(0)
                        else:
                              b_review.append(0)
                        s_review.append(rev[-1])
            print(nam.text, ad_f.text, vari.text, sep=',' )
            try:
                  wait.until(EC.element_to_be_clickable(driver.find_element(By.CLASS_NAME, "wyJ1s")))
            except:
                  time.sleep(1)
            driver.find_element(By.CLASS_NAME, "wyJ1s").click()
                      
            driver.switch_to.default_content()
            driver.switch_to.frame('searchIframe')
        
            driver.execute_script("arguments[0].scrollIntoView(false);",lis[count])
            if count2 == count:
                  
                  mok = driver.find_element(By.CLASS_NAME,'Ryr1F')  # 검색결과 목록
                  lis = mok.find_elements(By.TAG_NAME,'li') # 목록에서 순서별로 찾을 준비
                  count2 += 10
            else:
                  pass
            count += 1
            if (count+1 > len(lis)) & (count3 == 5): # 더이상 목록을 갱신할 수 없고 페이지가 5페이지일 경우 다음 구로 넘어감
                  break
            elif count+1 > len(lis) :   # 목록갱신하는 파트
                  try:
                        
                        driver.switch_to.default_content()
                        time.sleep(1)
                        driver.switch_to.frame('searchIframe')
                        n_p = driver.find_element(By.CLASS_NAME,'zRM9F')
                        n_b = n_p.find_elements(By.TAG_NAME,'a')
                        n_b[-1].click()
                        count = 0
                        count2 = 9
                        count3 += 1
                        time.sleep(1.5)
                        mok = driver.find_element(By.CLASS_NAME,'Ryr1F')  # 검색결과 목록
                        lis = mok.find_elements(By.TAG_NAME,'li') # 목록에서 순서별로 찾을 준비
                  except:
                        

                        break
      df = pd.DataFrame({'가게명':name,'평점':jum_su,'분류':var,'주소':add,'방문자리뷰':v_review,'블로그리뷰':b_review,'리뷰총합':s_review,'지역':gu_l})
      df.to_csv(f'./{i}.csv')

자연산 해담일식 대게마을,서울 강남구 언주로93길 12 1층 해담일식,일식당
뱅뱅막국수,서울 강남구 도곡로 112 서한빌딩, 1층,국수
바비레드 강남본점,서울 강남구 봉은사로6길 39 바비레드,이탈리아음식
땀땀,서울 강남구 강남대로98길 12-5,베트남음식
고수닭갈비,서울 강남구 강남대로96길 12,닭갈비
명가우육면 선릉역점,서울 강남구 선릉로90길 56 1층 106호,중식당


# 특이사항들
1. 네이버 지도는 특성상 기본검색결과를 나타내는 searchIframe, 검색결과에 클릭해서 추가로 상세정보를 띄워주는 entryIframe, 눈에 바로 보이지는 않지만
   기본틀을 구성하고 있는 default_content frame으로 구성되어 있다.
   이로 인해 다른 웹페이지처럼 로딩이 완료됬다고 해서 곧 바로 element 검색을 시행할 경우 찾을 수 없다는 오류가 발생한다.
   (element오류 이미지)
    따라서 driver.switch_to 함수를 이용해 반드시 기본틀 frame을 거쳐서 이동을 해야 element를 확보할 수 있다.
2. 검색 결과를 나타내는 목록은 개발자모드에서 볼 때는 수십개의 element가 표기되지만 실제 정보는 스크롤내려 10개마다 추가되어 갱신되기 때문에
    처음 for문에 목록의 길이만큼 반복시킬경우 10번밖에 반복되지 않아, for문을 사용할 수 없다.
    따라서 갱신되는 목록에서 다음목록 값이 있을 동안은 계속해서 반복되는 while문을 사용한다.
    (리스트인식 오류 이미지)
3. 만료이슈..... 컴퓨터차이, 회선차이 등 기타이슈로 인해 검색도중 페이지 정보가 만료되어 검색불가능해 지는 경우가 있었다.
    (페이지만료 오류 이미지)

In [ ]:
# gu = ['관악구']
# for i in gu:


#       driver.switch_to.default_content()

#       mat = driver.find_element(By.CLASS_NAME,'input_box')
#       mat.click()
#       sear = mat.find_element(By.CLASS_NAME,'input_search')
#       sear.clear()


#       sear.send_keys(i+' 맛집')
#       sear.send_keys(Keys.RETURN)
#       driver.implicitly_wait(3)

#       driver.switch_to.frame('searchIframe')
#       time.sleep(2)
#       mok = driver.find_element(By.CLASS_NAME,'Ryr1F')  # 검색결과 목록
#       lis = mok.find_elements(By.TAG_NAME,'li') # 목록에서 순서별로 찾을 준비
#       count = 0
#       count2 = 9
#       count3 = 1
#       n = 1





#       while lis[count] != 'null':    

          
#             num.append(n) # 데이터 분석을 용이하게 하기위한 넘버링
#             n += 1 #넘버링 카운터 증가
#             gu_l.append(i)
#             time.sleep(1.5)

#             title = lis[count].find_element(By.CLASS_NAME,'place_bluelink.TYaxT')
#             title.click()
#             driver.implicitly_wait(3)
#             driver.switch_to.default_content() # switch_to.default_content()로 전환해야 frame_in iframe을 제대로 잡을 수 있다. 
#             time.sleep(1.5)
#             driver.switch_to.frame("entryIframe") # 검색결과에서 세부사항을 보기위한 entryframe으로 이동
            
#             time.sleep(1.5)
#             nam = driver.find_element(By.CLASS_NAME,'GHAhO') #가게이름 
#             name.append(nam.text)
#             vari = driver.find_element(By.CLASS_NAME,'lnJFt') #식당종류
#             var.append(vari.text)

#             ad_f = driver.find_element(By.CLASS_NAME,'LDgIH') #주소
#             add.append(ad_f.text)

#             bul = driver.find_element(By.CLASS_NAME,'dAsGb') # 리뷰전체
#             bul_l = bul.find_elements(By.TAG_NAME,'a') # 방문자와 블로그 리뷰 분리
      

#             if 'class="PXMot LXIwF"' in bul.get_attribute('innerHTML'): # 별점이 있을경우
#                   jum = bul.find_element(By.CLASS_NAME,'PXMot.LXIwF') #별점 element 의 클래스네임
#                   point = jum.get_attribute('innerText').split() #별점만 분리
#                   jum_su.append(float(point[1])) # 소수형으로 변환하여 저장
#                   try:
                        
#                         rev1 = bul_l[0].get_attribute('innerText').split()  # 방문자 리뷰
#                         rev1[-1] = int(rev1[-1].replace(',','')) # 천단위가 넘어갈 경우 쉼표빼고 int형으로 변환
#                         v_review.append(rev1[-1]) # 방문자 리뷰 점수 추가
#                         rev2 = bul_l[1].get_attribute('innerText').split() #  블로그 리뷰 점수
                        
#                         rev2[-1] = int(rev2[-1].replace(',','')) # 형변환
#                         b_review.append(rev2[-1]) # 블로그 리뷰점수 추가
#                         s_review.append(rev1[-1]+rev2[-1]) #방문자와 블로그 리뷰 수를 더해서 총 리뷰수를 계산
#                   except:
#                         rev = bul_l[0].get_attribute('innerText').split()
#                         rev[-1] = int(rev[-1].replace(',',''))
                    
#                         if v_review[n-2] == rev[-1]:
#                               b_review.append(0)
#                         elif b_review[n-2] == rev[-1]:
#                               v_review.append(0)
                      
#                         s_review.append(rev[-1])
                        
            
                  
#             else:
#                   try: # 블로그 리뷰나 방문자 리뷰 둘중 하나가 없을 경우 무시하고 하나값을 총 리뷰수로 취급
                        
#                         jum_su.append(0) # 별점이 없을경우 0점
#                         rev1 = bul_l[0].get_attribute('innerText').split()
#                         rev1[-1] = int(rev1[-1].replace(',',''))
#                         v_review.append(rev1[-1])
#                         rev2 = bul_l[1].get_attribute('innerText').split()
                        
#                         rev2[-1] = int(rev2[-1].replace(',',''))
#                         b_review.append(rev2[-1])
#                         s_review.append(rev1[-1]+rev2[-1])
                  
#                   except:
#                         rev = bul_l[0].get_attribute('innerText').split()
#                         rev[-1] = int(rev[-1].replace(',',''))
                    
#                         if v_review[n-2] == rev[-1]:
#                               b_review.append(0)
#                         elif b_review[n-2] == rev[-1]:
#                               v_review.append(0)
#                         s_review.append(rev[-1])

       

#             driver.switch_to.default_content()
#             driver.switch_to.frame('searchIframe')
#             time.sleep(1)
#             driver.execute_script("arguments[0].scrollIntoView(false);",lis[count])
            
#             if count2 == count:
#                   time.sleep(2)
#                   mok = driver.find_element(By.CLASS_NAME,'Ryr1F')  # 검색결과 목록
#                   lis = mok.find_elements(By.TAG_NAME,'li') # 목록에서 순서별로 찾을 준비
#                   count2 += 10
            

#             else:
#                   time.sleep(1)
            
#             count += 1

      



#             if (count+1 > len(lis)) & (count3 == 3):
#                   break



#             elif count+1 > len(lis) :
                  
                        
#                   try:
#                         print('다음페이지')
#                         driver.switch_to.default_content()
#                         driver.switch_to.frame('searchIframe')
#                         n_p = driver.find_element(By.CLASS_NAME,'zRM9F')
#                         n_b = n_p.find_elements(By.TAG_NAME,'a')
#                         n_b[-1].click()
#                         count = 0
#                         count2 = 9
#                         count3 += 1
#                         time.sleep(3)
#                         mok = driver.find_element(By.CLASS_NAME,'Ryr1F')  # 검색결과 목록
#                         lis = mok.find_elements(By.TAG_NAME,'li') # 목록에서 순서별로 찾을 준비
      

#                   except:
#                         print('종료')
#                         break
        

                        






In [ ]:
# account_add.execute('revoke all on project.* from "jh"@"%"')
# account_add.execute('revoke all on project.* from "yj"@"%"')
# account_add.execute('revoke all on project.* from "mh"@"%"')
# account_add.execute('drop user "jh"@"%"')
# account_add.execute('drop user "yj"@"%"')
# account_add.execute('drop user "mh"@"%"')
# account_add.execute('drop database project')